# Time series classification
Time series classification (TSC)

Using data from timeseriesclassification.com
TODO add reference to Dau et al. 2018

The SonyAIBORobotSurface dataset was donated to the TSC website by Manuela Veloso and Douglas Vail of Carnegie Mellon University. Their paper on robot surface detection is
https://www.cs.cmu.edu/~mmv/papers/04iav-doug.pdf
TODO - get full reference.


# Load Python packages
Import the Python packages that we will need.

In [ ]:
from pathlib import Path
import time

import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model

# User settings

In [ ]:
load_from_web = True

# General settings

In [ ]:
sns.set_style('whitegrid')

# Load the data
TODO introduce the dataset and what it is used for.

The dataset has been split into two, balanced, datasets. One for model development and one for our final test to evaluate the finished model.

In [ ]:
if load_from_web:
    url = 'https://raw.githubusercontent.com/Withington/deepscent/master/data/SonyAIBORobotSurface1_IoC/SonyAIBORobotSurface1_IoC_DEV.txt'
    robot_df = pd.read_csv(url, sep='\t', header=None)
    print('Loaded from', url)
    robot_data = robot_df.values
else:
    data_dir = '../../data'
    data_name = 'SonyAIBORobotSurface1_IoC'
    data_filename = data_dir+'/'+data_name+'/'+data_name+'_DEV.txt'
    robot_data = np.loadtxt(Path(data_filename))
    print('Loaded from', data_filename)
print('The shape of robot_data is', robot_data.shape)
print('robot_data:', robot_data)

In [ ]:
y_dev = robot_data[:,0]
x_dev = robot_data[:,1:]
print('The shape of x_dev is', x_dev.shape)
print('The shape of y_dev is', y_dev.shape)

In [ ]:
# Change from classes 1 and 2 to classes 0 and 1
y_dev = (y_dev - y_dev.min())/(y_dev.max()-y_dev.min())

In [ ]:
print('Number of samples of class 0', (y_dev == 0).sum())
print('Number of samples of class 1', (y_dev == 1).sum())
y_dev_df = pd.DataFrame(y_dev)
y_dev_df[0].value_counts().plot(kind='bar')

## Plot the data

In [ ]:
sample_number = 3 ### CHANGE PARAMETER HERE ###
plt.plot(x_dev[sample_number], label='category'+str(y_dev[sample_number]))
plt.legend(loc='upper right', frameon=False)

In [ ]:
sample_a = 0 ### CHANGE PARAMETER HERE ###
sample_b = 3 ### CHANGE PARAMETER HERE ###
plt.plot(x_dev[sample_a], label='category'+str(y_dev[sample_a]))
plt.plot(x_dev[sample_b], label='category'+str(y_dev[sample_b]))
plt.legend(loc='upper right', frameon=False)

# Split the development dataset into training and test datasets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_dev, y_dev, test_size=100, random_state=21, stratify=y_dev)
print('The shape of train_data is', x_train.shape)
print('The shape of test_data is', x_test.shape)
print('Training data:')
print('Number of samples of class 0', (y_train == 0).sum())
print('Number of samples of class 1', (y_train == 1).sum())
print('Test data:')
print('Number of samples of class 0', (y_test == 0).sum())
print('Number of samples of class 1', (y_test == 1).sum())

# Pre-process the data

In [ ]:
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std) 
x_test = (x_test - x_train_mean)/(x_train_std)

print('x_train_mean', x_train_mean)
print('x_train_std', x_train_std)

In [ ]:
sample_a = 0 ### CHANGE PARAMETER HERE ###
sample_b = 4 ### CHANGE PARAMETER HERE ###
plt.plot(x_train[sample_a], label='category'+str(y_train[sample_a]))
plt.plot(x_train[sample_b], label='category'+str(y_train[sample_b]))
plt.legend(loc='upper right', frameon=False)

# MLP 1
Create an multilayer perceptron (MLP). This first MLP is small.

In [ ]:
input_shape = x_train.shape[1:]

In [ ]:
n0 = 16
x = Input(shape=(input_shape), name='MLP1InputLayer')
y = Dense(n0, activation='relu', name='Dense010')(x)
# Output layer
out = Dense(1, activation='sigmoid', name='OutputLayer')(y)

# Build model
model_mlp1 = Model(x, out)
print(model_mlp1.summary())

## Understanding the number of parameters
TODO - exercise around the calculation that arrives at the number of parameters in each layer.

In [ ]:
print(70*16+16)
print(16+1)

## Select an optimizer and compile the model

In [ ]:
optimizer = keras.optimizers.Adam()
model_mlp1.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# TODO - can we access TensorBoard on colab? If so, add tensorboard callback

## Train MLP 1

In [ ]:
batch_size = 5
epochs = 50

In [ ]:
start = time.time()
hist = model_mlp1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose=1)
end = time.time()
log = pd.DataFrame(hist.history) 
print('Training complete in', round(end-start), 'seconds')

In [ ]:
print('The first five rows in log are')
log.head()

In [ ]:
log[['loss', 'val_loss']].plot()
# TODO add axes labels, etc

In [ ]:
log[['acc', 'val_acc']].plot()

# Make predictions using MLP 1
Classify the data using MLP 1

In [ ]:
result = model_mlp1.evaluate(x_test, y_test, batch_size=batch_size)
print('Validation accuracy is', result[1])

In [ ]:
y_probability = model_mlp1.predict_on_batch(x_test)
y_predicted_class = np.round(y_probability).flatten()

In [ ]:
sample = 3
print('The probability that sample', sample, 'belongs to class 1 is', y_probability[sample][0])
print('The model classifies sample', sample, 'as class', y_predicted_class[sample])
print('The true class of sample', sample, 'is class', y_test[sample])

In [ ]:
sample_a = 0 ### CHANGE PARAMETER HERE ###
sample_b = 3 ### CHANGE PARAMETER HERE ###
plt.plot(x_test[sample_a], label='True:'+str(y_test[sample_a])+' Pred:'+str(y_predicted_class[sample_a]))
plt.plot(x_test[sample_b], label='True:'+str(y_test[sample_b])+' Pred:'+str(y_predicted_class[sample_b]))
plt.legend(loc='upper right', frameon=False)

# MLP 2
This time we will create a function that builds our model.

In [ ]:
def build_model():
    num = 16
    x = Input(shape=(input_shape), name='MLP2InputLayer')
    y = Dropout(0.1,name='Drop010')(x)
    y = Dense(num, activation='relu', name='Dense010')(y)
    y = Dropout(0.2,name='Drop020')(y)
    y = Dense(num, activation='relu', name='Dense020')(y)
    y = Dropout(0.2,name='Drop030')(y)
    y = Dense(num, activation='relu', name='Dense021')(y)
    y = Dropout(0.3,name='Drop040')(y)
    out = Dense(1, activation='sigmoid', name='OutputLayer')(y)

    # Build model and compile the model
    model = Model(x, out)
    optimizer = keras.optimizers.Adam()
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
    
model = build_model()

In [ ]:
batch_size = 5
epochs = 50
start = time.time()
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose=1)
end = time.time()
log = pd.DataFrame(hist.history) 
print('Training complete in', round(end-start), 'seconds')

In [ ]:
log[['loss', 'val_loss']].plot()

In [ ]:
log[['acc', 'val_acc']].plot()
model.evaluate(x_test, y_test, batch_size=batch_size)
print('Validation accuracy is', result[1])

# k-fold cross validation

In [ ]:
model = build_model()
#model.summary()
sample_a = 0 ### CHANGE PARAMETER HERE ###
sample_b = 3 ### CHANGE PARAMETER HERE ###
plt.plot(x_dev[sample_a], label='True:'+str(y_dev[sample_a]))
plt.plot(x_dev[sample_b], label='True:'+str(y_dev[sample_b]))
plt.legend(loc='upper right', frameon=False)

In [ ]:
k = 3
batch_size = 10
epochs = 2
kfold = RepeatedStratifiedKFold(n_splits=k, n_repeats=1, random_state=76)
count = 0
val_acc = list()
for train, test in kfold.split(x_dev, y_dev):
    x_train, y_train, x_test, y_test = x_dev[train], y_dev[train], x_dev[test], y_dev[test]
    # Normalise the data
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std) 
    x_test = (x_test - x_train_mean)/(x_train_std)
    # Build and train a model
    model = build_model()
    start = time.time()
    hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), verbose=1)
    end = time.time()
    log = pd.DataFrame(hist.history) 
    print('Training of fold', count, 'complete in', round(end-start), 'seconds')
    val_acc.append(log.iloc[-1]['val_acc'])
    count = count + 1

val_acc = pd.DataFrame(val_acc, columns=['val_acc'])
print(val_acc)

## Plot the k-fold cross validation results

In [ ]:
ax = sns.boxplot(data=val_acc)
ax = sns.swarmplot(data=val_acc, color='black')
print('Validation accuracy mean and sample standard deviation', val_acc['val_acc'].mean(), val_acc['val_acc'].std())